In [ ]:
# 1. Import required modules
from anomalib.data import MVTecAD
from anomalib.deploy import ExportType
from anomalib.engine import Engine
from anomalib.models import Patchcore
from anomalib.data import Folder
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def albu_adapter(aug):
    """Wrap an Albumentations Compose so it accepts (image) and returns a Tensor.
       Converts PIL/Tensor to NumPy HxWxC uint8 before calling Albumentations.
    """
    def _call(image):
        # 1) Normalize input type to NumPy HxWxC
        if isinstance(image, np.ndarray):
            img = image
            # If CHW, convert to HWC
            if img.ndim == 3 and img.shape[0] in (1, 3, 4) and img.shape[2] not in (1, 3, 4):
                img = np.transpose(img, (1, 2, 0))
        elif isinstance(image, Image.Image):
            img = np.array(image)  # PIL -> HWC uint8
        elif isinstance(image, torch.Tensor):
            arr = image.detach().cpu().numpy()
            # Assume CHW; convert to HWC
            if arr.ndim == 3 and arr.shape[0] in (1, 3, 4):
                arr = np.transpose(arr, (1, 2, 0))
            # If float in [0,1], convert to uint8 0..255 for Albumentations Normalize defaults
            if arr.dtype != np.uint8:
                arr = np.clip(arr, 0, 1) * 255.0
                arr = arr.astype(np.uint8)
            img = arr
        else:
            raise TypeError(f"Unsupported image type: {type(image)}. Expected numpy, PIL, or torch.Tensor.")

        # Ensure 3rd dimension exists for grayscale
        if img.ndim == 2:
            img = img[..., None]

        # 2) Call Albumentations with named argument
        out = aug(image=img)
        # out["image"] is a torch.Tensor thanks to ToTensorV2()
        return out["image"]
    return _call

# --- Define transforms (Albumentations still) ---
train_aug = A.Compose([
    A.Resize(256, 256),
    # Optional robustifying augs for field images:
    # A.HorizontalFlip(p=0.5),
    # A.ColorJitter(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # expects uint8 input
    ToTensorV2(),
])

eval_aug = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

train_tf = albu_adapter(train_aug)
eval_tf  = albu_adapter(eval_aug)


In [ ]:
dm = Folder(
    name="blattodea_anomaly_detection",
    root="drive/MyDrive/Colab Notebooks/AnomalySpiders",
    normal_dir="normal",
    abnormal_dir="abnormal",
    train_augmentations=train_tf,
    val_augmentations=eval_tf,
    test_augmentations=eval_tf,
    train_batch_size=16,
    eval_batch_size=16,
    num_workers=0,  # set to 0 for easier debugging; increase later
)